In [1]:
from Process import PlateReader
import pandas as pd

### Step 1: Load Sample Information
* Ideally will be in the form of a dataframe. This dataframe should be standardized in the original documentation where we originally made the sample. 
* Either load locally or pull from the google team drive (still working on fixing pulling from Drive)

In [42]:
sample_info = pd.DataFrame(data=[0, 5,10,40, 0, 5,10,40], columns = ['Sonication Time']) #pd.read_csv(r"04_16_21_Yt")
sample_info['84mM Cd:TOP volf'] = [0, 0.75, 0.75, 0.75, 0, 0.75, 0.75, 0.75]
sample_info['1M Se:TOP volf'] = [0, 0.25, 0.25, 0.25, 0, 0.25, 0.25, 0.25]
sample_info['Ocatdecence volf'] = [100, 0, 0, 0, 100, 0, 0, 0]
sample_info

,Sonication Time,84mM Cd:TOP volf,1M Se:TOP volf,Ocatdecence volf
0,0,0.00,0.00,100
1,5,0.75,0.25,0
2,10,0.75,0.25,0
3,40,0.75,0.25,0
4,0,0.00,0.00,100
5,5,0.75,0.25,0
6,10,0.75,0.25,0
7,40,0.75,0.25,0


### Step 2: Load UV-Vis Data
* ***If loading from plate reader***: Once the plate has been read, allow for exporting to an excel sheet and reformat for easy upload and merging.
    * The plate reader should automatically ask you if you would like to export to excel. When running multiple plates you can just keep selecting export and it will automatically add the new run as an additional sheet to the originally generated excel sheet. 
    * To format for easy loading:
        1. Create a new sheet in the same exported excel file and name it something simple (i.e. sheet1). 
        2. Copy the data from the sheet and add it to the newly created sheet. Make sure to paste in the data as "only values". 
        3. Repeat for every single exported sheet (i.e. every plate)
         
* Sometimes when using the plate reader the scanning protocol scans the whole plate versus only the wells of interest. This can be changed, however it is common to forget. If you do forget to do this, make sure to trim the dataframe to match the same length/order of wells as the sample info loaded in Step 1.

* ***If loading from single cuvette reader***: 
    * Still documenting.

***Step 2a:*** Provide the path of the excel sheet for the plate reader. With this you will also need to provide the **Sheet Name** of for correct excel sheet to be extracted. Since multiple plates are expected to be used at once, provide the Sheet names as a ***list of string names***. This will return a list of dataframes of each plate in the order the sheet names provided.

In [34]:
path = r"C:\Users\Edwin\Desktop\05_04_21_FL.xlsx"
plate_names = ['85', '95']
plate_dfs = PlateReader.extract_plates(path, plate_names)

***Step 2b:*** With the dataframes from ***Step2a***, given that all the plates read originally were done using the same protcol, then we can merge these dataframes into one. The only thing we need to edit is when originally formatting the excel sheets, since we copied the wavelength row in each sheet it only needs to be present once. A way to avoid to this is to copy it only in the first plate sheet you are using. 

Should be length of sample_info + 1 (will remove the wavelength column shortly).

In [35]:
merged_df = PlateReader.merge_wavelength_dfs(plate_dfs)
merged_df

,0,1,2,3,4,5,6,7,8,9,...,57,58,59,60,61,62,63,64,65,66
Wavelength,370,375,380,385,390,395,400,405,410,415,...,655,660,665,670,675,680,685,690,695,700
E1,OVRFLW,68274,11886,11619,11416,10998,10119,9141,8333,7629,...,26,89,26,32,26,24,25,1,0,45
E6,OVRFLW,47381,8756,8692,8482,8281,7747,7034,6512,6058,...,9197,8822,8267,7681,7425,7094,6651,6705,5986,6104
E12,OVRFLW,59877,*7535*,7399,7190,6948,6380,5724,5087,4636,...,19549,18092,17153,16424,15688,14412,13094,12915,12688,11980
H1,OVRFLW,10652,1663,1633,1567,1480,1299,1093,1009,1146,...,14343,13295,11865,11597,10601,9373,9294,8283,7793,7709
E1,OVRFLW,OVRFLW,32535,31807,31223,29981,27610,24986,22587,20739,...,110,91,71,79,142,94,50,0,289,144
E6,OVRFLW,OVRFLW,19111,18936,18610,18095,16870,15422,14099,13245,...,19831,19165,17656,16760,15919,14740,13556,12873,13440,12734
E12,OVRFLW,OVRFLW,15614,15528,15207,14500,13479,11994,10618,9827,...,39556,38126,35122,33830,31895,30986,28192,26418,24654,23741
H1,OVRFLW,20847,3305,3199,3110,2863,2553,2111,2024,2300,...,27586,25924,23660,21591,21364,19470,17766,16460,15471,13952


***Step 2c:*** Sometimes when samples are extremely turbid or have some sort of material like foam or cream the plate reader absorbance is overflowed and returns as 'OVRFLW'. This will prevent you from handling the entire set of data, so what can be done is replace the 'OVRFLW' values with a holder number. It is reccomended you select a very high holder number so you can track and isolate it easier if needed. 

In [48]:
merged_df = PlateReader.detect_ovflw(merged_df, holder = 30)
merged_df

,0,1,2,3,4,5,6,7,8,9,...,57,58,59,60,61,62,63,64,65,66
Wavelength,370,375,380,385,390,395,400,405,410,415,...,655,660,665,670,675,680,685,690,695,700
E1,30,68274,11886,11619,11416,10998,10119,9141,8333,7629,...,26,89,26,32,26,24,25,1,0,45
E6,30,47381,8756,8692,8482,8281,7747,7034,6512,6058,...,9197,8822,8267,7681,7425,7094,6651,6705,5986,6104
E12,30,59877,*7535*,7399,7190,6948,6380,5724,5087,4636,...,19549,18092,17153,16424,15688,14412,13094,12915,12688,11980
H1,30,10652,1663,1633,1567,1480,1299,1093,1009,1146,...,14343,13295,11865,11597,10601,9373,9294,8283,7793,7709
E1,30,30,32535,31807,31223,29981,27610,24986,22587,20739,...,110,91,71,79,142,94,50,0,289,144
E6,30,30,19111,18936,18610,18095,16870,15422,14099,13245,...,19831,19165,17656,16760,15919,14740,13556,12873,13440,12734
E12,30,30,15614,15528,15207,14500,13479,11994,10618,9827,...,39556,38126,35122,33830,31895,30986,28192,26418,24654,23741
H1,30,20847,3305,3199,3110,2863,2553,2111,2024,2300,...,27586,25924,23660,21591,21364,19470,17766,16460,15471,13952


# Step 3: Merge with Sample Info Dataframe

The first step to merge to sample info dataframe is to format the plate reader dataframe to have the wavelengths as the column names versus as an another row. We can do this by isolating the Wavelength row, adding the appropiate unit the string of each element of the wavlength row and reassigning this new list as the column names. We finalize this by dropping the wavlength row.

In [49]:
platereader_df = PlateReader.rehead_wavelengths(merged_df)
# platereader_df.insert(0, 'Gain', [85,85,85,85, 95, 95, 95, 95])
platereader_df

,370nm,375nm,380nm,385nm,390nm,395nm,400nm,405nm,410nm,415nm,...,655nm,660nm,665nm,670nm,675nm,680nm,685nm,690nm,695nm,700nm
E1,30,68274,11886,11619,11416,10998,10119,9141,8333,7629,...,26,89,26,32,26,24,25,1,0,45
E6,30,47381,8756,8692,8482,8281,7747,7034,6512,6058,...,9197,8822,8267,7681,7425,7094,6651,6705,5986,6104
E12,30,59877,*7535*,7399,7190,6948,6380,5724,5087,4636,...,19549,18092,17153,16424,15688,14412,13094,12915,12688,11980
H1,30,10652,1663,1633,1567,1480,1299,1093,1009,1146,...,14343,13295,11865,11597,10601,9373,9294,8283,7793,7709
E1,30,30,32535,31807,31223,29981,27610,24986,22587,20739,...,110,91,71,79,142,94,50,0,289,144
E6,30,30,19111,18936,18610,18095,16870,15422,14099,13245,...,19831,19165,17656,16760,15919,14740,13556,12873,13440,12734
E12,30,30,15614,15528,15207,14500,13479,11994,10618,9827,...,39556,38126,35122,33830,31895,30986,28192,26418,24654,23741
H1,30,20847,3305,3199,3110,2863,2553,2111,2024,2300,...,27586,25924,23660,21591,21364,19470,17766,16460,15471,13952


Next we reset both indexes of the two dataframes we are to merge and concatnate them together. We drop and reset indexes to ensure any past dependency is forgotten, so you must make sure your sample info dataframe and plate reader dataframe are ordered correctly where row 1...n of one dataframe referes to row 1...n of the other. 

In [50]:
sample_info.reset_index(drop=True, inplace=True)
platereader_df.reset_index(drop=True, inplace=True)
combined_df = pd.concat([sample_info, platereader_df], axis = 1) # this is called combine hotdog. 
combined_df.insert(0, 'Gain', [85,85,85,85, 95, 95, 95, 95])
combined_df

,Gain,Sonication Time,84mM Cd:TOP volf,1M Se:TOP volf,Ocatdecence volf,370nm,375nm,380nm,385nm,390nm,...,655nm,660nm,665nm,670nm,675nm,680nm,685nm,690nm,695nm,700nm
0,85,0,0.00,0.00,100,30,68274,11886,11619,11416,...,26,89,26,32,26,24,25,1,0,45
1,85,5,0.75,0.25,0,30,47381,8756,8692,8482,...,9197,8822,8267,7681,7425,7094,6651,6705,5986,6104
2,85,10,0.75,0.25,0,30,59877,*7535*,7399,7190,...,19549,18092,17153,16424,15688,14412,13094,12915,12688,11980
3,85,40,0.75,0.25,0,30,10652,1663,1633,1567,...,14343,13295,11865,11597,10601,9373,9294,8283,7793,7709
4,95,0,0.00,0.00,100,30,30,32535,31807,31223,...,110,91,71,79,142,94,50,0,289,144
5,95,5,0.75,0.25,0,30,30,19111,18936,18610,...,19831,19165,17656,16760,15919,14740,13556,12873,13440,12734
6,95,10,0.75,0.25,0,30,30,15614,15528,15207,...,39556,38126,35122,33830,31895,30986,28192,26418,24654,23741
7,95,40,0.75,0.25,0,30,20847,3305,3199,3110,...,27586,25924,23660,21591,21364,19470,17766,16460,15471,13952


# Step 4: Format Dataframe as csv and Upload to Drive.
We simply call a method of the dataframe while ensuring to drop the index (again to remove and previous dependency). 

***If uploading from desktop***: Just grab and upload like any other file.

***If uploading from jupyter notebook***: Save the csv anywhere with a name that is not relevant, reccomended is something like "Temp File" as you will be uploading it to google drive under a different name. Then use the Gdrive module of functions from the Prepare package. Read the README in this package to understand how to use it, it will require a certain setup of credentials. 


In [51]:
temp_hold_path = r"05_03_21_CdTOP_Blanked_FL"
combined_df_csv = combined_df.to_csv(temp_hold_path, index=False)

,Total Volume,CdOl 84mM volpct,TOP 1mM volpct,ZnOl 84mM volpct,Time Sonicated,200nm,205nm,210nm,215nm,220nm,...,855nm,860nm,865nm,870nm,875nm,880nm,885nm,890nm,895nm,900nm
0,5.0,75.0,25.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,5.0,75.0,25.0,0.0,0.0,-0.007,-0.015,-0.007,-0.007,-0.006,...,-0.006,-0.006,-0.006,-0.006,-0.005,-0.006,-0.005,-0.005,-0.005,-0.005
2,5.0,75.0,24.5,0.5,0.0,0.047,0.042,0.052,0.051,0.045,...,-0.007,-0.007,-0.007,-0.007,-0.006,-0.006,-0.006,-0.005,-0.005,-0.005
3,5.0,75.0,24.5,0.5,0.0,0.036,0.036,0.038,0.042,0.040,...,-0.006,-0.006,-0.006,-0.006,-0.004,-0.004,-0.004,-0.003,-0.003,-0.003
4,5.0,75.0,24.0,1.0,0.0,0.027,0.028,0.032,0.038,0.031,...,-0.004,-0.004,-0.004,-0.004,-0.002,-0.003,-0.002,-0.002,-0.002,-0.001
5,5.0,75.0,24.0,1.0,0.0,0.038,0.044,0.046,0.049,0.048,...,-0.005,-0.005,-0.005,-0.005,-0.003,-0.004,-0.003,-0.003,-0.003,-0.002
6,5.0,75.0,20.0,5.0,0.0,0.042,0.044,0.045,0.050,0.048,...,-0.008,-0.008,-0.008,-0.008,-0.006,-0.007,-0.006,-0.006,-0.006,-0.006
7,5.0,75.0,20.0,5.0,0.0,0.046,0.051,0.055,0.055,0.052,...,-0.007,-0.007,-0.007,-0.007,-0.005,-0.006,-0.005,-0.005,-0.005,-0.004
8,5.0,75.0,25.0,0.0,10.0,0.144,0.134,0.148,0.159,0.152,...,-0.005,-0.005,-0.005,-0.005,-0.004,-0.004,-0.003,-0.003,-0.003,-0.003
9,5.0,75.0,25.0,0.0,10.0,0.111,0.101,0.116,0.121,0.116,...,-0.009,-0.008,-0.009,-0.009,-0.008,-0.009,-0.007,-0.007,-0.007,-0.007
